In [3]:
%%javascript

UsageError: %%javascript is a cell magic, but the cell body is empty.


# Print all cell output

In [4]:
m=5
a=6
a
m

5

In [5]:
from IPython.core.interactiveshell import InteractiveShell
# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

In [6]:
a
m

6

5

In [ ]:
# to restore previous case
from IPython.core.interactiveshell import InteractiveShell

# pretty print only the last output of the cell
InteractiveShell.ast_node_interactivity = "last_expr"

To make this behavior consistent across all notebooks edit: ~/.ipython/profile_default/ipython_config.py:

c = get_config()

Run all nodes interactively

c.InteractiveShell.ast_node_interactivity = "all"

## Suppressing Output
The side-effect of this change is that some libraries will start spewing a lot of noise (e.g. matplotlib). To fix that add ; at the end of the lines whose output you don’t want to be displayed.

Example:

fig, axes = plt.subplots(2, 1, figsize=(20, 20))

axes[0].set_ylim(-5, 5);

Without using ; it outputs:

Out[53]: (-5, 5)

However the hack doesn’t seem to work when the code line that outputs something is part of some loop, in which case the workaround of assigning to _ does the trick:

fig, axes = plt.subplots(2, 1, figsize=(20, 20))

for i in [1,2]:

    _ = axes[i].set_ylim(-5, 5)

## To Skip A Cell From Running (e.g. work in progress)

Add at the top of the cell:

%%script false

some multiline code 

that you want to skip for a time being 

(e.g. work in progress) 
without commenting out / deleting cell 
goes here

## Rmarkdown in output

In [7]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

printmd("**bold text**")

**bold text**

In [8]:
#use html
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

printmd("**bold and blue**", color="blue")

<span style='color:blue'>**bold and blue**</span>

## Restart and Run All fix
My browser is too slow and “Restart and Run All” only manages to restart, but fails to “Run All”. This adds the missing delay and as a bonus a shortcut Meta-F (to complement Alt-F=Run All ignoring errors). Runs from ‘Command Mode’ (i.e. may need to hit Esc first).

// Meta-F: "Restart and Run All" slow delay fix + shortcut [Command mode]
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Meta-F', {
    help: 'Restart And Run All',
    help_index: 'zz',
    handler: function (event) {
      Jupyter.notebook.kernel.restart();
      restartTime = 2000 // decrease this if you have a fast computer
      setTimeout(function(){ Jupyter.notebook.execute_all_cells(); }, restartTime);
      return false;
    }
});

## Control notebook resource allocation
Sometimes notebooks take a way more resources than available, causing undesired behavior, like a hanging system.

There are various ways to limit system resource usage, I decided to try the modern cgroups to limit memory usage in this case.

sudo cgcreate -a stas:stas -t stas:stas -g memory:JupyterGroup

replace stas with your setup’s user:group (usually yourusername:yourusername).

Then I decided to give each notebook max of 15GB RAM and 25GB Total (RAM+SWAP):

sudo echo $(( 15000 * 1024 * 1024 )) > /sys/fs/cgroup/memory/JupyterGroup/memory.limit_in_bytes

sudo echo $(( 10000 * 1024 * 1024 )) > /sys/fs/cgroup/memory/JupyterGroup/memory.kmem.max_usage_in_bytes
Now I start the notebook as:

cgexec -g memory:JupyterGroup jupyter notebook

a greedy kernel now gets terminated if it consumes more than what I allocated, without crashing/overloading my system.

Note that filenames under /sys/fs/cgroup/memory/JupyterGroup/ might be slightly different on your system. You may have to check the documentation.

I also needed to install the following packages, before I was able to run this:

sudo apt-get install cgroup-bin cgroup-lite cgroup-tools cgroupfs-mount libcgroup1

## Following the currently executing cell mode
Won’t it be nice to be able to watch the progress of the notebook run hands off? Now you can:

Add the following in ~/.jupyter/custom/custom.js and reload the notebooks you’re running:

/*
 In Command mode Meta-[ toggles Follow Exec Cell mode, Meta-] turns it off.

 To adjust the behavior you can adjust the arguments:
 * behavior: One of "auto", "instant", or "smooth". Defaults to "auto". Defines the transition animation.
 * block:    One of "start", "center", "end", or "nearest". Defaults to "center".
 * inline:   One of "start", "center", "end", or "nearest". Defaults to "nearest".
 https://developer.mozilla.org/en-US/docs/Web/API/Element/scrollIntoView
*/
function scrollIntoRunningCell(evt, data) {
    $('.running')[0].scrollIntoView({behavior: 'smooth', inline: 'center'});
}

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Meta-[', {
    help: 'Follow Executing Cell On',
    help_index: 'zz',
    handler: function (event) {
        Jupyter.notebook.events.on('finished_execute.CodeCell', scrollIntoRunningCell);
        //console.log("Follow Executing Cell On")
        return false;
    }
});

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Meta-]', {
    help: 'Follow Executing Cell Off',
    help_index: 'zz',
    handler: function (event) {
        Jupyter.notebook.events.off('finished_execute.CodeCell', scrollIntoRunningCell);
        //console.log("Follow Executing Cell Off")
        return false;
    }
});

Now in Command Mode (when cell in focus has a blue box around it and not green, or hit Esc to toggle mode), hit Meta-[ to get the currently run cell stay in the middle of the screen, hit Meta-] to return to normal behavior.

If this is not working, debug this setup by uncommenting console.log() calls and watch your browser Developer Tools’ Console to check that custom.js got loaded without errors and that the shortcuts got registered and the handler is activated. Sometimes you need to restart jupyter notebook, but most of the time tab-reload works.

If you just want to jump once to the current executing cell use Alt-I after you add the following to ~/.jupyter/custom/custom.js and reload the notebooks you’re running:

// Alt-I: Go to Running cell shortcut [Command mode]
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Alt-I', {
    help : 'Go to Running cell',
    help_index : 'zz',
    handler : function (event) {
        setTimeout(function() {
            // Find running cell and click the first one
            if ($('.running').length > 0) {
                //alert("found running cell");
                $('.running')[0].scrollIntoView();
            }}, 250);
        return false;
    }
});
Caveat: for it to work - the sections should all be uncollapsed - otherwise it won’t know to go into a collapsed section.

You can adjust the activation shortcut keys to your liking.

Remember that all 3 shortcuts will only work in the Command mode (see above for figuring that out).

This has been tested to work with jupyter notebook 5.6.0 with python 3.6.6.